### Imports and settings

In [1]:
import mne
import numpy as np
import os
import matplotlib.pyplot as plt
import random

from mne import Epochs, compute_covariance, find_events, make_ad_hoc_cov
from mne.datasets import sample
from mne.simulation import (
    add_ecg,
    add_eog,
    add_noise,
    simulate_raw,
    simulate_sparse_stc,)

### Load real data as template

In [23]:
epochs_filepath = '/Users/Bella/Desktop/Grosenick_Lab/eeg_patient_data/mdd_dlpfc/subject9_m191_dlpfc_58/mne/m191_day1_hplp_epo.fif'
raw_filepath = '/Users/Bella/Desktop/Grosenick_Lab/eeg_patient_data/mdd_dlpfc/subject9_m191_dlpfc_58/m191_dlpfc_day1/m191_dlpfc_day1_treatment_20230424_021417.mff'
fwd_path = '/Users/Bella/mne_data/MNE-sample-data/MEG/sample/sample_audvis-meg-eeg-oct-6-fwd.fif'
epoch_fif = mne.read_epochs(epochs_filepath)
epoch_fif.set_eeg_reference(projection=True)
raw = mne.io.read_raw_egi(raw_filepath, preload=True)
raw_events = mne.find_events(raw, stim_channel="DIN1")

Reading /Users/Bella/Desktop/Grosenick_Lab/eeg_patient_data/mdd_dlpfc/subject9_m191_dlpfc_58/mne/m191_day1_hplp_epo.fif ...
    Found the data of interest:
        t =       0.00 ...      95.00 ms
        0 CTF compensation matrices available
Not setting metadata
600 matching events found
No baseline correction applied
0 projection items activated
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Reading EGI MFF Header from /Users/Bella/Desktop/Grosenick_Lab/eeg_patient_data/mdd_dlpfc/subject9_m191_dlpfc_58/m191_dlpfc_day1/m191_dlpfc_day1_treatment_20230424_021417.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...
Reading 0 ... 720329  =      0.000 ...   720.329 secs...
600 events found on stim channel DIN1
Event IDs: [1]


### Define times array for simulated data (one epoch)

In [24]:
random.seed(0)

# Sampling frequency (in downsampled data)
sfreq = 200

# Define time points for creating array. Define sine and cosine waves
times = np.linspace(0, 0.1, 20, endpoint=False)  # (0 sec, 0.1 sec, 20 samples/0.1 sec)
print(times)
alpha = np.sin(20 * np.pi * times) 
print(alpha)

[0.    0.005 0.01  0.015 0.02  0.025 0.03  0.035 0.04  0.045 0.05  0.055
 0.06  0.065 0.07  0.075 0.08  0.085 0.09  0.095]
[ 0.00000000e+00  3.09016994e-01  5.87785252e-01  8.09016994e-01
  9.51056516e-01  1.00000000e+00  9.51056516e-01  8.09016994e-01
  5.87785252e-01  3.09016994e-01  1.22464680e-16 -3.09016994e-01
 -5.87785252e-01 -8.09016994e-01 -9.51056516e-01 -1.00000000e+00
 -9.51056516e-01 -8.09016994e-01 -5.87785252e-01 -3.09016994e-01]


### Define waveforms

In [25]:
# Define wave frequencies
# Conversion of frequency in Hz to angular frequency: Hz * 2pi = radians/sec
# Multiplying this by the times vector scales the values over time

### Does it matter which are sine vs cosine waves? Just changes initalization of wave in time
alpha = np.sin(20 * np.pi * times)  # Alpha 10 Hz
gammalow = np.cos(60 * np.pi * times)  # Low-range gamma 30 Hz
gammahigh = np.sin(120 * np.pi * times)  # High-range gamma 60 Hz
betahigh = np.cos(42 * np.pi * times)  # High-range beta 21 Hz
betalow = np.sin(26 * np.pi * times)  # Low-range Beta 13 Hz
theta = np.cos(10 * np.pi * times)  # Theta 5 Hz
delta = np.sin(4 * np.pi * times)  # Delta 2 Hz
deltaslow = np.cos(0.6 * np.pi * times)  # Delta ultra slow 0.3 Hz

### Create data for 600 epochs, 256 channels

In [26]:
# Repeat each frequency component array 120 times
# 5 epochs * 120 = 600 total epochs for each wave | 20 time points in times
# Wave array shape = (600, 20)

# set a new random.seed in between each 
alpha_data = np.array([random.uniform(0, 1) * alpha for _ in range(600)])
gammalow_data = np.array([random.uniform(0, 1) * gammalow for _ in range(600)])
gammahigh_data = np.array([random.uniform(0, 1) * gammahigh for _ in range(600)])
betahigh_data = np.array([random.uniform(0, 1) * betahigh for _ in range(600)])
betalow_data = np.array([random.uniform(0, 1) * betalow for _ in range(600)])
theta_data = np.array([random.uniform(0, 1) * theta for _ in range(600)])
delta_data = np.array([random.uniform(0, 1) * delta for _ in range(600)])
deltaslow_data = np.array([random.uniform(0, 1) * deltaslow for _ in range(600)])

print(alpha_data.shape)

# Stack data along the second axis to get shape (600, 256, 20) where (epochs, channels, time points)
# 8 wave arrays * 32 = 256 channels
data_array = np.stack([alpha_data, gammalow_data, gammahigh_data, betahigh_data, betalow_data, theta_data, delta_data, deltaslow_data] * 32, axis=1)
print(data_array.shape)

# Create info object
info = mne.create_info([f'EEG{n:03}' for n in range(1, 257)], ch_types='misc', sfreq=sfreq)

# Convert array to epochs using raw events as template
simulated_epochs = mne.EpochsArray(data_array, info, tmin=0, baseline= (0,0), events=raw_events)

(600, 20)
(600, 256, 20)
Not setting metadata
600 matching events found
Applying baseline correction (mode: mean)
0 projection items activated


In [27]:
simulated_epochs.plot(picks="misc", title="simulated data")

Using pyopengl with version 3.1.6
